<a href="https://colab.research.google.com/github/anjalinagel12/Google-colab-notebook/blob/master/QuerySearchFromMultipleDocuments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all the required libraries.

In [1]:
!pip install num2words

     |████████████████████████████████| 102kB 5.6MB/s 


In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [3]:
!unzip nlp_hackathon_search.zip

unzip:  cannot find or open nlp_hackathon_search.zip, nlp_hackathon_search.zip.zip or nlp_hackathon_search.zip.ZIP.


In [0]:
df = pd.read_csv('nlp_hackathon_search.csv')

In [59]:
df.tail(50)

,document,question,title,document_id
62349,The law of the United States comprises many le...,The constitution set boundaries for case law t...,Law_of_the_United_States,42919d881bfe11ea8f2b656571b1b549
62350,"In Paris, Chopin encountered artists and other...",What was the name of the poet Chopin became ac...,Frédéric_Chopin,427c2cbe1bfe11ea8f2b656571b1b549
62351,The war started badly for the US and UN. North...,What South Korean city did the US and ROK forc...,Military_history_of_the_United_States,429478d21bfe11ea8f2b656571b1b549
62352,NaN,NaN,NaN,NaN
62353,NaN,NaN,NaN,NaN
62354,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",NaN,NaN,NaN
62355,NaN,NaN,NaN,NaN
62356,"<html xmlns=""http://www.w3.org/1999/xhtml"" >",NaN,NaN,NaN
62357,<head><title>,NaN,NaN,NaN
62358,\tFileDownload,NaN,NaN,NaN


In [28]:
df.isnull().sum()

document        5
question       47
title          47
document_id    47
dtype: int64

Drop all Missing Rows

In [0]:
df = df.dropna()

In [30]:
df.tail()

,document,question,title,document_id
62347,A government report covered by the Guardian in...,What are the children of some families in the ...,Ministry_of_Defence_(United_Kingdom),426ae62a1bfe11ea8f2b656571b1b549
62348,Luftwaffe policy at this point was primarily t...,What did the Fliegerkorps X concentrate on?,The_Blitz,4280fb361bfe11ea8f2b656571b1b549
62349,The law of the United States comprises many le...,The constitution set boundaries for case law t...,Law_of_the_United_States,42919d881bfe11ea8f2b656571b1b549
62350,"In Paris, Chopin encountered artists and other...",What was the name of the poet Chopin became ac...,Frédéric_Chopin,427c2cbe1bfe11ea8f2b656571b1b549
62351,The war started badly for the US and UN. North...,What South Korean city did the US and ROK forc...,Military_history_of_the_United_States,429478d21bfe11ea8f2b656571b1b549


In [31]:
df.isnull().sum()

document       0
question       0
title          0
document_id    0
dtype: int64

 Function to convert to Lower Case

In [0]:
def convert_lower_case(data):
    return np.char.lower(data)

Function to remove Stop Words

In [0]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

Function to remove Puntuation

In [0]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

Function to remove Apostrophe

In [0]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

Applying Stemming

In [0]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

Convert Everything to Numbers

In [0]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [0]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [40]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
preprocess_document = []
for doc in df['document'].tolist():
    preprocess_document.append(word_tokenize(str(preprocess(doc))))


tfidf vectorizer

In [0]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [0]:
doc = 0

tf_idf = {}
N = len(preprocess_document)
for i in range(N):
    
    tokens = preprocess_document[i]
    
    counter = Counter(tokens + preprocess_document[i])
    words_count = len(tokens + preprocess_document[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df1 = doc_freq(token)
        idf = np.log((N+1)/(df1+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [45]:

tf_idf

{(0, '1up'): 0.20445494565607064,
 (0, 'acclaim'): 0.20445494565607064,
 (0, 'aggreg'): 0.20445494565607064,
 (0, 'averag'): 0.20445494565607064,
 (0, 'call'): 0.20445494565607064,
 (0, 'com'): 0.20445494565607064,
 (0, 'commerci'): 0.20445494565607064,
 (0, 'comput'): 0.20445494565607064,
 (0, 'creat'): 0.20445494565607064,
 (0, 'critic'): 0.20445494565607064,
 (0, 'electron'): 0.20445494565607064,
 (0, 'ever'): 0.20445494565607064,
 (0, 'five'): 0.613364836968212,
 (0, 'game'): 0.8178197826242826,
 (0, 'gamecub'): 0.20445494565607064,
 (0, 'gamerank'): 0.20445494565607064,
 (0, 'gamespi'): 0.20445494565607064,
 (0, 'gamesradar'): 0.20445494565607064,
 (0, 'gametrail'): 0.20445494565607064,
 (0, 'greatest'): 0.20445494565607064,
 (0, 'inform'): 0.20445494565607064,
 (0, 'major'): 0.20445494565607064,
 (0, 'metacrit'): 0.20445494565607064,
 (0, 'monthli'): 0.20445494565607064,
 (0, 'nineti'): 0.8178197826242826,
 (0, 'one'): 0.20445494565607064,
 (0, 'perfect'): 0.20445494565607064,
 (

In [0]:
alpha = 0.3

In [0]:
for i in tf_idf:
    tf_idf[i] *= alpha

In [48]:
tf_idf

{(0, '1up'): 0.06133648369682119,
 (0, 'acclaim'): 0.06133648369682119,
 (0, 'aggreg'): 0.06133648369682119,
 (0, 'averag'): 0.06133648369682119,
 (0, 'call'): 0.06133648369682119,
 (0, 'com'): 0.06133648369682119,
 (0, 'commerci'): 0.06133648369682119,
 (0, 'comput'): 0.06133648369682119,
 (0, 'creat'): 0.06133648369682119,
 (0, 'critic'): 0.06133648369682119,
 (0, 'electron'): 0.06133648369682119,
 (0, 'ever'): 0.06133648369682119,
 (0, 'five'): 0.1840094510904636,
 (0, 'game'): 0.24534593478728475,
 (0, 'gamecub'): 0.06133648369682119,
 (0, 'gamerank'): 0.06133648369682119,
 (0, 'gamespi'): 0.06133648369682119,
 (0, 'gamesradar'): 0.06133648369682119,
 (0, 'gametrail'): 0.06133648369682119,
 (0, 'greatest'): 0.06133648369682119,
 (0, 'inform'): 0.06133648369682119,
 (0, 'major'): 0.06133648369682119,
 (0, 'metacrit'): 0.06133648369682119,
 (0, 'monthli'): 0.06133648369682119,
 (0, 'nineti'): 0.24534593478728475,
 (0, 'one'): 0.06133648369682119,
 (0, 'perfect'): 0.06133648369682119,

In [49]:
df['document'].tolist()

['Twilight Princess was released to universal critical acclaim and commercial success. It received perfect scores from major publications such as 1UP.com, Computer and Video Games, Electronic Gaming Monthly, Game Informer, GamesRadar, and GameSpy. On the review aggregators GameRankings and Metacritic, Twilight Princess has average scores of 95% and 95 for the Wii version and scores of 95% and 96 for the GameCube version. GameTrailers in their review called it one of the greatest games ever created.',
 'On 16 August 1960, Cyprus attained independence after the Zürich and London Agreement between the United Kingdom, Greece and Turkey. Cyprus had a total population of 573,566; of whom 442,138 (77.1%) were Greeks, 104,320 (18.2%) Turks, and 27,108 (4.7%) others  The UK retained the two Sovereign Base Areas of Akrotiri and Dhekelia, while government posts and public offices were allocated by ethnic quotas, giving the minority Turkish Cypriots a permanent veto, 30% in parliament and administ

In [0]:
def preprocess_score(search_string):
    preprocessed_query = preprocess(search_string)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", search_string)
    print("Matching results of query are:")
    print("")
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    print(query_weights[:10])
    for i in query_weights[:10]:
        l.append(i[1])
    
    return sum(l)

In [61]:
search_string ="What was the origin of Hindu philosophy??"
preprocess_score(search_string)


Query: What was the origin of Hindu philosophy??
Matching results of query are:


[(6110, 0.40765170703118087), (11690, 0.40765170703118087), (44248, 0.40765170703118087), (49788, 0.40765170703118087), (1751, 0.4067577339894458), (15866, 0.4067577339894458), (46454, 0.4067577339894458), (54852, 0.4067577339894458), (6383, 0.2760141766356954), (22924, 0.2760141766356954)]


3.8096661173538977

In [62]:
search_string = "What kind of scores did Twilight Princess receive from many video game review sources?"
preprocess_score(search_string)


Query: What kind of scores did Twilight Princess receive from many video game review sources?
Matching results of query are:


[(0, 0.920047255452318), (6124, 0.920047255452318), (17937, 0.920047255452318), (44558, 0.920047255452318), (60331, 0.920047255452318), (60657, 0.920047255452318), (14707, 0.7429166623465445), (18367, 0.7429166623465445), (18441, 0.7429166623465445), (27775, 0.7429166623465445)]


8.491950182100087

In [63]:
search_string ="What is the date that Cyrpus attained independence?"
preprocess_score(search_string)


Query: What is the date that Cyrpus attained independence?
Matching results of query are:


[(6987, 0.4215489243163347), (9287, 0.4215489243163347), (40145, 0.4215489243163347), (44810, 0.4215489243163347), (8361, 0.25478231689448805), (30098, 0.25478231689448805), (47536, 0.25478231689448805), (7038, 0.23658357997345317), (7096, 0.23658357997345317), (16785, 0.23658357997345317)]


3.160293387869163

In [64]:
search_string ="How many seats are in the Provincial Assembly??"
preprocess_score(search_string)


Query: How many seats are in the Provincial Assembly??
Matching results of query are:


[(15088, 0.48177019921866826), (51459, 0.48177019921866826), (57742, 0.48177019921866826), (59774, 0.48177019921866826), (13215, 0.3680189021809272), (24144, 0.3680189021809272), (42295, 0.3680189021809272), (46750, 0.3680189021809272), (181, 0.3105159487151573), (23540, 0.3105159487151573)]


4.020188303028696